In [22]:
import os
import tensorflow as tf
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [10]:
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, 'dataset', '3 - normalized')

dataset = CandyDataset(dataset_dir)
dataset.split_dataset()

In [11]:
train_set_data = dataset.get_images_data(dataset.data_split['train'])
test_set_data  = dataset.get_images_data(dataset.data_split['test'])
validation_set_data  = dataset.get_images_data(dataset.data_split['val'])

def prepare_data(data, num_classes):
    # Normalizar imagens e criar arrays
    images = np.array([d['image'] for d in data]) / 255.0 
    labels = to_categorical(np.array([d['category_id'] - 1 for d in data]), num_classes)
    masks = np.array([d['mask'] for d in data]) 
    bboxes = np.array([d['bbox'] for d in data]) 

    return images, masks, labels, bboxes

num_classes = 10  

# Preparar os dados
X_train, masks_train, y_train, bboxes_train = prepare_data(train_set_data, num_classes)
X_val, masks_val, y_val, bboxes_val = prepare_data(validation_set_data, num_classes)
X_test, masks_test, y_test, bboxes_test = prepare_data(test_set_data, num_classes)


In [12]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [13]:
tf.config.list_physical_devices('GPU')

[]

In [17]:
kernel = (5, 5)
pool = (2, 2)
num_classes = 10

inputs = Input(shape=(256, 256, 3))

x = Conv2D(16, (3, 3), padding='same', activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Classification output
x_class = Flatten()(x)
x_class = Dense(128, activation='relu')(x_class)
x_class = Dropout(0.5)(x_class)
outputs_class = Dense(num_classes, activation='softmax', name='classification')(x_class)

# Segmentation output
x_seg = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
outputs_seg = Conv2D(num_classes, (1, 1), activation='softmax', name='segmentation')(x_seg)

# Bounding boxes output
x_bbox = Flatten()(x)
outputs_bbox = Dense(4, activation='linear', name='bounding_boxes')(x_bbox)

model = Model(inputs=inputs, outputs=[outputs_class, outputs_seg, outputs_bbox])


In [24]:
model.compile(optimizer=Adam(), 
              loss={'dense': 'categorical_crossentropy', 'conv2d_2': 'sparse_categorical_crossentropy'}, 
              metrics={'dense': 'accuracy', 'conv2d_2': 'accuracy'})

In [25]:
history = model.fit(X_train, 
                    {'dense': y_train, 'conv2d_2': masks_train}, 
                    validation_data=(X_val, {'dense': y_val, 'conv2d_2': masks_val}), 
                    epochs=10, 
                    batch_size=32)

Epoch 1/10



KeyboardInterrupt



In [ ]:
test_loss, test_acc = model.evaluate(X_test, {'dense': y_test, 'conv2d_2': masks_test, 'dense_2': bboxes_test})
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot loss
plt.figure(figsize=(12, 6))

# Plot total loss and individual losses
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Total Loss')
plt.plot(history.history['dense_loss'], label='Classification Loss')
plt.plot(history.history['conv2d_2_loss'], label='Segmentation Loss')
plt.plot(history.history['dense_2_loss'], label='Bounding Boxes Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['dense_accuracy'], label='Classification Accuracy')
plt.plot(history.history['conv2d_2_accuracy'], label='Segmentation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

# Plot segmentation loss
plt.subplot(1, 2, 1)
plt.plot(history.history['conv2d_2_loss'], label='Segmentation Loss')
plt.title('Segmentation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot bounding boxes loss
plt.subplot(1, 2, 2)
plt.plot(history.history['dense_2_loss'], label='Bounding Boxes Loss')
plt.title('Bounding Boxes Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()